2일차에 이어서

### 각 이미지 파일에 대한 인식도 HOG로 변경 

In [1]:
import cv2
import numpy as np
import os
from PIL import Image
import dlib

In [5]:
hog_face_detector = dlib.get_frontal_face_detector()
recognizer = cv2.face.LBPHFaceRecognizer_create()

Face_ID = -1
prev_person_name =""
y_ID = []
x_train = []

Face_Images = os.path.join(os.getcwd(), "Face_Images2")
print(Face_Images)

for root, dirs, files in os.walk(Face_Images): # 파일 목록 가져오기
    for file in files:
        if file.endswith('jpeg') or file.endswith('jpg') or file.endswith('png'): # 이미지 파일 필터링
            path = os.path.join(root, file)
            person_name = os.path.basename(root)
            print(path, person_name)
            
            if prev_person_name != person_name : # 이름이 바뀌었는지 확인
                Face_ID += 1
                prev_person_name = person_name
                
            img = cv2.imread(path) # 이미지 파일 가져오기
            gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = hog_face_detector(gray_image) # 얼굴 찾기
            
            print(Face_ID, faces)
            
            for face_detection in faces:
                x, y, w, h = face_detection.left(), face_detection.top(), face_detection.right(), face_detection.bottom()
                roi = gray_image[y:y+h, x:x+w]
                x_train.append(roi)
                y_ID.append(Face_ID)
                
                recognizer.train(x_train, np.array(y_ID))
                recognizer.save('face-trainner.yml')

c:\Users\main\workspace\vision\2025-06-11\Face_Images2
c:\Users\main\workspace\vision\2025-06-11\Face_Images2\Ka\ka1.jpg Ka
0 rectangles[[(69, 53) (141, 125)]]
c:\Users\main\workspace\vision\2025-06-11\Face_Images2\Ka\ka2.jpg Ka
0 rectangles[[(93, 45) (179, 131)]]
c:\Users\main\workspace\vision\2025-06-11\Face_Images2\Ka\ka3.jpg Ka
0 rectangles[[(61, 53) (133, 125)]]
c:\Users\main\workspace\vision\2025-06-11\Face_Images2\Ka\ka4.jpg Ka
0 rectangles[[(69, 69) (141, 141)]]
c:\Users\main\workspace\vision\2025-06-11\Face_Images2\Ka\ka5.jpg Ka
0 rectangles[[(45, 64) (131, 150)]]
c:\Users\main\workspace\vision\2025-06-11\Face_Images2\Win\win1.jpg Win
1 rectangles[[(53, 45) (125, 117)]]
c:\Users\main\workspace\vision\2025-06-11\Face_Images2\Win\win2.jpg Win
1 rectangles[[(83, 54) (170, 141)]]
c:\Users\main\workspace\vision\2025-06-11\Face_Images2\Win\win3.jpg Win
1 rectangles[[(45, 74) (131, 160)]]
c:\Users\main\workspace\vision\2025-06-11\Face_Images2\Win\win4.jpg Win
1 rectangles[[(53, 45) (

In [9]:
labels = ['Ka', 'Win']

hog_face_detector = dlib.get_frontal_face_detector()
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('face-trainner.yml')

image_list = []

test_images = os.path.join(os.getcwd(), 'test_list2')

for root, dirs, files in os.walk(test_images):
    for file in files:
        if file.endswith('jpg') or file.endswith('jpeg') or file.endswith('png'):
            image_path = os.path.join(test_images, file)
            print(image_path)
            image_list.append(cv2.imread(image_path))
        
for img in image_list:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = hog_face_detector(gray, 1)
    
    for face_detection in faces:
        x, y, w, h = face_detection.left(), face_detection.top(), face_detection.right(), face_detection.bottom()
        roi_gray = gray[y:y+h, x:x+w]
        
        id_, conf = recognizer.predict(roi_gray)
        print(labels[id_], conf)
        
        if conf >=50 :
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[id_]
            cv2.putText(img, name, (x,y), font, 1, (0,0,255), 2)
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)

c:\Users\main\workspace\vision\2025-06-11\test_list2\ka_test1.jpg
c:\Users\main\workspace\vision\2025-06-11\test_list2\ka_test2.jpg
c:\Users\main\workspace\vision\2025-06-11\test_list2\win_test1.jpg
c:\Users\main\workspace\vision\2025-06-11\test_list2\win_test2.jpg
Ka 93.8054934759378
Win 144.69140239273813
Win 100.42224921714629
Ka 144.7741850166738


In [17]:
import cv2
import time

labels = ['Ka', 'Win']
webcam = cv2.VideoCapture(0)

face_cascade = cv2.CascadeClassifier('./files/haarcascade_frontalface_default.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('face-trainner.yml')

if not webcam.isOpened():
    print('Could not open webcam')
    exit()
    
while webcam.isOpened():
    status, frame = webcam.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    start = time.time()
    faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=4)
    end = time.time()
    
    for (x,y,w,h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        id_, conf = recognizer.predict(roi_gray)
        print(labels[id_], conf)
        
        if conf >=50 :
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[id_]
            cv2.putText(frame, name, (x,y), font, 1, (0,0,255), 2)
            org = (50, 100)
            text = f'{end - start:.4f} seconds'
            cv2.putText(frame, text, org, font, 1, (255,0,0), 2)
            cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
            
    if status:
        cv2.imshow('test', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcam.release()
cv2.destroyAllWindows()

Win 96.90153546291755
Win 95.40264396400063
Win 95.84155655211659
Win 96.05302445051012
Win 95.41641834409295
Win 94.87953457357222
Win 95.86791817366179
Win 95.51382197331768
Win 96.26975892490044
Win 95.5852327403467
Win 95.2821160833169
Win 95.84048237127752
Win 96.56773649139949
Win 95.85782875803608
Win 96.54258619386323
Win 95.75982855825731
Win 95.45547646472829
Win 95.8053957878513
Win 94.8145000308764
Win 95.85715585602911
Win 95.54977787647546
Win 96.17085355307141
Win 97.36190158120908
Win 96.60639160178974
Win 97.36893733348057
Win 97.03874762604555
Win 97.10325781071104
Win 96.47847422787801
Win 96.13973228369116
Win 97.55810785506183
Win 95.88343978430039
Win 95.39730066068239
Win 95.88648048741479
Win 95.69404355400026
Win 95.42306854230624
Win 96.45617284359038
Win 96.39978264822138
Win 96.66649937004786
Win 95.50337289316967
Win 94.99956057147172
Win 95.35213204910717
Win 94.7059209383263
Win 95.41354341897683
Win 95.41354341897683
Win 96.5723221776665
Win 96.577176925

cnn 으로 변경

In [ ]:
cnn_face_detector = dlib.cnn_face_detection_model_v1('./files/mmod_human_face_detector.dat')
recognizer = cv2.face.LBPHFaceRecognizer_create()

Face_ID = -1
prev_person_name =""
y_ID = []
x_train = []

Face_Images = os.path.join(os.getcwd(), "Face_Images2")
print(Face_Images)

for root, dirs, files in os.walk(Face_Images): # 파일 목록 가져오기
    for file in files:
        if file.endswith('jpeg') or file.endswith('jpg') or file.endswith('png'): # 이미지 파일 필터링
            path = os.path.join(root, file)
            person_name = os.path.basename(root)
            print(path, person_name)
            
            if prev_person_name != person_name : # 이름이 바뀌었는지 확인
                Face_ID += 1
                prev_person_name = person_name
                
            img = cv2.imread(path) # 이미지 파일 가져오기
            gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = cnn_face_detector(gray_image) # 얼굴 찾기
            
            print(Face_ID, faces)
            
            for face_detection in faces:
                x, y, w, h = face_detection.rect.left(), face_detection.rect.top(), face_detection.rect.right(), face_detection.rect.bottom()
                roi = gray_image[y:y+h, x:x+w]
                x_train.append(roi)
                y_ID.append(Face_ID)
                
            recognizer.train(x_train, np.array(y_ID))
            recognizer.save('face-trainner2.yml')

c:\Users\main\workspace\vision\2025-06-11\Face_Images2
c:\Users\main\workspace\vision\2025-06-11\Face_Images2\Ka\ka1.jpg Ka
0 <_dlib_pybind11.mmod_rectangles object at 0x000002113AFB9B30>
c:\Users\main\workspace\vision\2025-06-11\Face_Images2\Ka\ka2.jpg Ka
0 <_dlib_pybind11.mmod_rectangles object at 0x0000021148F14430>
c:\Users\main\workspace\vision\2025-06-11\Face_Images2\Ka\ka3.jpg Ka
0 <_dlib_pybind11.mmod_rectangles object at 0x0000021148F14830>
c:\Users\main\workspace\vision\2025-06-11\Face_Images2\Ka\ka4.jpg Ka
0 <_dlib_pybind11.mmod_rectangles object at 0x0000021148F14070>
c:\Users\main\workspace\vision\2025-06-11\Face_Images2\Ka\ka5.jpg Ka
0 <_dlib_pybind11.mmod_rectangles object at 0x0000021148F14830>
c:\Users\main\workspace\vision\2025-06-11\Face_Images2\Win\win1.jpg Win
1 <_dlib_pybind11.mmod_rectangles object at 0x0000021148F14030>
c:\Users\main\workspace\vision\2025-06-11\Face_Images2\Win\win2.jpg Win
1 <_dlib_pybind11.mmod_rectangles object at 0x0000021148F144F0>
c:\Users

In [21]:
labels = ['Ka', 'Win']

cnn_face_detector = dlib.cnn_face_detection_model_v1('./files/mmod_human_face_detector.dat')
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('face-trainner2.yml')

image_list = []

test_images = os.path.join(os.getcwd(), 'test_list2')

for root, dirs, files in os.walk(test_images):
    for file in files:
        if file.endswith('jpg') or file.endswith('jpeg') or file.endswith('png'):
            image_path = os.path.join(test_images, file)
            print(image_path)
            image_list.append(cv2.imread(image_path))
        
for img in image_list:
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = cnn_face_detector(gray, 1)
    
    for face_detection in faces:
        x, y, w, h = face_detection.rect.left(), face_detection.rect.top(), face_detection.rect.right(), face_detection.rect.bottom()
        roi_gray = gray[y:y+h, x:x+w]
        
        id_, conf = recognizer.predict(roi_gray)
        print(labels[id_], conf)
        
        if conf >=50 :
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[id_]
            cv2.putText(img, name, (x,y), font, 1, (0,0,255), 2)
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)

c:\Users\main\workspace\vision\2025-06-11\test_list2\ka_test1.jpg
c:\Users\main\workspace\vision\2025-06-11\test_list2\ka_test2.jpg
c:\Users\main\workspace\vision\2025-06-11\test_list2\win_test1.jpg
c:\Users\main\workspace\vision\2025-06-11\test_list2\win_test2.jpg
Win 91.83134350872263
Win 91.83134350872263
Win 91.83134350872263
Win 91.83134350872263


In [ ]:
import cv2
import numpy as np
import os
from PIL import Image

labels = ['Ka', 'Win']

face_cascade = cv2.CascadeClassifier('./files/haarcascade_frontalface_default.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("face-trainner2.yml")

image_list = []
test_images = os.path.join(os.getcwd(), "test_list2")

for root, dirs, files in os.walk(test_images):
    for file in files:
        if file.endswith('jpeg') or file.endswith('jpg') or file.endswith('png'): # 이미지 파일 필터링
            image_path = os.path.join(test_images, file)
            print(image_path)
            image_list.append(cv2.imread(image_path))
            
for img in image_list:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)
    
    for (x,y,w,h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        
        id_, conf = recognizer.predict(roi_gray)
        print(labels[id_], conf)
        
        if conf >= 50 :
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[id_]
            cv2.putText(img, name, (x,y), font, 1, (0,0,255),2)
            cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0),2)
            
    cv2.imshow("Preview", img)
    if cv2.waitKey(0) >= 0:
        continue
    
cv2.destroyAllWindows()

In [22]:
import cv2
import time

labels = ['Ka', 'Win']
webcam = cv2.VideoCapture(0)

face_cascade = cv2.CascadeClassifier('./files/haarcascade_frontalface_default.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('face-trainner2.yml')

if not webcam.isOpened():
    print('Could not open webcam')
    exit()
    
while webcam.isOpened():
    status, frame = webcam.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    start = time.time()
    faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=4)
    end = time.time()
    
    for (x,y,w,h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        id_, conf = recognizer.predict(roi_gray)
        print(labels[id_], conf)
        
        if conf >=50 :
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[id_]
            cv2.putText(frame, name, (x,y), font, 1, (0,0,255), 2)
            org = (50, 100)
            text = f'{end - start:.4f} seconds'
            cv2.putText(frame, text, org, font, 1, (255,0,0), 2)
            cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
            
    if status:
        cv2.imshow('test', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcam.release()
cv2.destroyAllWindows()

Win 100.10255082227867
Win 98.24840736218698
Win 98.94285074810868
Win 98.3918231530776
Win 99.07313669943177
Win 98.83969815138173
Win 99.19582953065037
Win 98.4012717169476
Win 98.43070103242364
Win 98.59611829369071
Win 99.00936981273313
Win 99.59607973614409
Win 100.01383378952879
Win 98.83199117620256
Win 99.45084931355814
Win 100.11499851563349
Win 103.73077238187805
Win 102.14757294814743
Win 102.99496776200043
Win 103.25602294998258
Win 100.82675097168193
Win 102.28443790473902
Win 101.6545090600354
Win 100.51019550900779
Win 101.55590128352416
Win 101.12215256416144
Win 101.99097929515843
Win 101.72511956333679
Win 102.33390746068993
Win 101.26376203775314
Win 101.17373919485328
Win 101.33895483754121
Win 101.17887733539315
Win 101.53078650720616
Win 102.54552847514105
Win 101.41032166384922
Win 100.95667311224982
Win 102.4825545830894
Win 100.74192436801899
Win 102.47968630284043
Win 101.73568026996085
Win 102.03133846220322
Win 99.3787031593668
Win 101.30896524919545
